# Problem 1

We saw how to solve TSPs in this module and in particular presented two approaches to encode a TSP as an integer linear program. In this problem, we will ask you to adapt the TSP solution to the related problem of $k$ Travelling Salespeople Problem ($k$-TSP).

Let $G$ be a complete graph with $n$ vertices that we will label $0, \ldots, n-1$ (keeping Python array indexing in mind). Our costs are specified using a matrix $C$ wherein $C_{i,j}$ is the cost of the edge from vertex $i$ to $j$ for $i \not= j$.

In this problem, we have $k \geq 1$ salespeople who must  start from vertex $0$ of the graph (presumably the location of the sales office) and together visit every location in the graph,  each returning back to vertex $0$. Each location must be visited exactly once by some salesperson in the team. Therefore, other than vertex $0$ (the start/end vertex of every salesperson's tour), no two salesperson tours have a vertex in common. Notice that for $k=1$, this is just the regular TSP problem we have studied. 

Also, all $k$ salespeople must be employed in the tour. In other words, if we have $k=3$ then each salesperson must start at $0$ and visit a sequence of one or more vertices and come back to $0$. No salesperson can be "idle".

## Example-1

Consider a graph with $5$ nodes and the following cost matrix:

$$ \begin{array}{c|ccccc}
  \text{Vertices} & 0 & 1 & 2 & 3 & 4 \\ 
   \hline
 0 & - & 3 & 4 & 3 & 5 \\ 
 1 & 1 & - & 2 & 4 & 1 \\ 
 2 & 2 & 1 & - & 5 & 4 \\ 
 3 & 1 & 1 & 5 & - & 4 \\ 
 4 & 2 & 1 & 3 & 5 & - \\ 
 \end{array}$$
 
 For instance $C_{2,3}$ the cost of edge from vertex $2$ to $3$ is $5$. The $-$ in the diagonal entries simply tells us that we do not care what goes in there since we do not have self-loops in the graph.
 
The optimal $2$-TSP tour for $k=2$ salespeople is shown below.
  - Salesperson # 1: $0 \rightarrow 2 \rightarrow 1 \rightarrow 4 \rightarrow 0$.
  - Salesperson # 2: $0 \rightarrow 3 \rightarrow 0$.
  
The total cost of the edges traversed by the two salespeople equals $12$.

For $k=3$, the optimal $3-$ TSP tour is as shown below.
  - Salesperson # 1: $0 \rightarrow 1 \rightarrow 4$, 
  - Salesperson # 2: $0 \rightarrow 2$, 
  - Salesperson # 3: $0 \rightarrow 3$.

The total cost is $16$.

The objective of this problem is to formulate an ILP using the MTZ approach.

### Problem 1A (MTZ approach)

We will use the same ILP setup as in our notes (see the notes on Exact Approaches to TSP that includes the ILP encodings we will use in this problem).
  - Decision variables $x_{i,j}$ for $i \not= j$ denoting that the tour traverses the edge from $i$ to $j$.
  - Time stamps $t_1, \ldots, t_{n-1}$. The start/end vertex $0$ does not get a time stamp.
  
Modify the MTZ approach to incorporate the fact that $k$ salespeople are going to traverse the graph.

#### (A) Degree Constraints

What do the new degree constraints look like? Think about how many edges in the tour will need to enter/leave each vertex? Note that you may have to treat vertex $0$ differently from the other vertices of the graph.

Your answer below is not graded. However you are encouraged to write it down and check with the answers to select problems provided at the end.

YOUR ANSWER HERE

### (B) Time Stamp Constraints 

Formulate the time stamp constraints for the $k$-TSP problem. Think about how you would need to change them to eliminate subtour.

Your answer below is not graded. However you are encouraged to write it down and check with the answers to select problems provided at the end.


YOUR ANSWER HERE

### (C) Implement

Complete the implementation of the function `k_tsp_mtz_encoding(n, k, cost_matrix)` below. It follows the same input convention as the code supplied in the notes. The input `n` denotes the size of the graph with vertices labeled `0`,.., `n-1`, `k` is the number of salespeople, and `cost_matrix` is a list of lists wherein `cost_matrix[i][j]` is the edge cost to go from `i` to `j` for `i != j`. Your code must avoid accessing `cost_matrix[i][i]` to avoid bugs. These entries will be supplied as `None` in the test cases.

Your code must return a list `lst` that has exactly $k$ lists in it, wherein `lst[j]` represents the locations visited by the $j^{th}$ salesperson. 

For the example above, for $k=2$, your code must return 
~~~
[ [0, 2, 1, 4], [0, 3] ]
~~~
For the example above, for $k=3$, your code must return
~~~
[ [0, 1, 4], [0, 2], [0, 3] ]
~~~

In [1]:
from pulp import *
def k_tsp_mtz_encoding(n, k, cost_matrix):
    # check inputs are OK
    assert 1 <= k < n
    assert len(cost_matrix) == n, f'Cost matrix is not {n}x{n}'
    assert all(len(cj) == n for cj in cost_matrix), f'Cost matrix is not {n}x{n}'
    prob = LpProblem('kTSP', LpMinimize)
    # finish your implementation here
    # your code must return a list of k-lists [ [0, i1, ..., il], [0, j1,...,jl],...] wherein 
    # the ith entry in our list of lists represents the 
    # tour undertaken by the ith salesperson
    # your code here
    dVars = [[[LpVariable(f'x{i}_{j}', lowBound=0, upBound=1, cat="Integer") for j in range(n)] for i in  range(n)],[LpVariable(f't{i}', lowBound=0, cat="Integer") for i in range(n)]]

    # Objective function
    prob += lpSum([cost_matrix[i][j] * dVars[0][i][j] if i!=j else None for i in range(n) for j in range(n)])

    # Constraints
    prob += lpSum([dVars[0][0][j] for j in range(1, n)]) == k
    prob += lpSum([dVars[0][j][0] for j in range(1, n)]) == k
    for i in range(1,n):
        prob += lpSum([dVars[0][i][j] for j in range(n)]) == 1
        prob += lpSum([dVars[0][j][i] for j in range(n)]) == 1

    for i in range(n):
        for j in range(n):
            if i!=j and i!=0 and j!=0:
                prob += dVars[1][i] - dVars[1][j] + dVars[0][i][j] - (n - 1) * (1 - dVars[0][i][j]) <= 0
            elif i == j:
                prob += dVars[0][i][j] == 0

    prob.solve()

    tours = []
    vertices = [i for i in range(1,n)] + [0]*k
    for i in range(k):
        j = 0
        tour = [0]
        while len(vertices) > 0:
            for l in vertices:
                if dVars[0][j][l].varValue == 1:
                    if l!=0: tour.append(l)
                    vertices.remove(l)
                    j = l
                    break
            if j == 0:
                tours.append(tour)
                break
    return tours

    raise NotImplementedError

In [2]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
k=2
all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) == k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 12) <= 0.001, f'Expected tour cost is 12, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 3 points')


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/alimohammed1624/.virtualenvs/temp/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/nz/m7btrx796jl1bvgth7q4y5wm0000gn/T/e85d0a2fe9794297be6c5fa398f1f0fc-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/nz/m7btrx796jl1bvgth7q4y5wm0000gn/T/e85d0a2fe9794297be6c5fa398f1f0fc-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 32 COLUMNS
At line 200 RHS
At line 228 BOUNDS
At line 258 ENDATA
Problem MODEL has 27 rows, 29 columns and 89 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 12 - 0.00 seconds
Cgl0002I 5 variables fixed
Cgl0003I 0 fixed, 4 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 22 rows, 24 columns (24 integer (20 of which binary)) and 76 elements
Cutoff increment increased from 1e-05 to 0.9999
Cb

In [3]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
k=3
all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) == k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 16) <= 0.001, f'Expected tour cost is 16, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 2 points')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/alimohammed1624/.virtualenvs/temp/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/nz/m7btrx796jl1bvgth7q4y5wm0000gn/T/82167f035dd2454997c5cbaa25cfb2c1-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/nz/m7btrx796jl1bvgth7q4y5wm0000gn/T/82167f035dd2454997c5cbaa25cfb2c1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 32 COLUMNS
At line 200 RHS
At line 228 BOUNDS
At line 258 ENDATA
Problem MODEL has 27 rows, 29 columns and 89 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 16 - 0.00 seconds
Cgl0002I 5 variables fixed
Cgl0003I 0 fixed, 4 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 13 rows, 16 columns (16 integer (12 of which binary)) and 48 elements
Cutoff increment increased from 1e-05 to 0.9999
Cb

In [4]:
cost_matrix = [ 
 [None, 1, 1, 1, 1, 1, 1, 1],
    [0, None, 1, 2, 1, 1, 1, 1],
    [1, 0, None, 1, 2, 2, 2, 1],
    [1, 2, 2, None, 0, 1, 2, 1],
    [1, 1, 1, 1, None, 1, 1, 1],
    [0,  1, 2, 1, 1, None, 1, 1],
    [1, 0,  1, 2, 2, 2,None, 1],
    [1, 2, 2, 0, 1, 2, 1, None],
]
n = 8
k = 2

all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) == k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 4) <= 0.001, f'Expected tour cost is 4, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 3 points')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/alimohammed1624/.virtualenvs/temp/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/nz/m7btrx796jl1bvgth7q4y5wm0000gn/T/34ff4b0fe0a5445f862df3ecda9d6895-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/nz/m7btrx796jl1bvgth7q4y5wm0000gn/T/34ff4b0fe0a5445f862df3ecda9d6895-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 71 COLUMNS
At line 524 RHS
At line 591 BOUNDS
At line 663 ENDATA
Problem MODEL has 66 rows, 71 columns and 260 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 4 - 0.00 seconds
Cgl0002I 8 variables fixed
Cgl0003I 0 fixed, 7 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 58 rows, 63 columns (63 integer (56 of which binary)) and 238 elements
Cutoff increment increased from 1e-05 to 0.9999
C

In [5]:
cost_matrix = [ 
 [None, 1, 1, 1, 1, 1, 1, 1],
    [0, None, 1, 2, 1, 1, 1, 1],
    [1, 0, None, 1, 2, 2, 2, 1],
    [1, 2, 2, None, 0, 1, 2, 1],
    [1, 1, 1, 1, None, 1, 1, 1],
    [0,  1, 2, 1, 1, None, 1, 1],
    [1, 0,  1, 2, 2, 2,None, 1],
    [1, 2, 2, 0, 1, 2, 1, None],
]
n = 8
k = 4

all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) == k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 6) <= 0.001, f'Expected tour cost is 6, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 2 points')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/alimohammed1624/.virtualenvs/temp/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/nz/m7btrx796jl1bvgth7q4y5wm0000gn/T/45ac2f2789834a6cbbd8119ba05376b1-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/nz/m7btrx796jl1bvgth7q4y5wm0000gn/T/45ac2f2789834a6cbbd8119ba05376b1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 71 COLUMNS
At line 524 RHS
At line 591 BOUNDS
At line 663 ENDATA
Problem MODEL has 66 rows, 71 columns and 260 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 6 - 0.00 seconds
Cgl0002I 8 variables fixed
Cgl0003I 0 fixed, 7 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 58 rows, 63 columns (63 integer (56 of which binary)) and 238 elements
Cutoff increment increased from 1e-05 to 0.9999
C

In [6]:
from random import uniform, randint

def create_cost(n):
    return [ [uniform(0, 5) if i != j else None for j in range(n)] for i in range(n)]

for trial in range(5):
    print(f'Trial # {trial}')
    n = randint(5, 11)
    k = randint(2, n//2)
    print(f' n= {n}, k={k}')
    cost_matrix = create_cost(n)
    print('cost_matrix = ')
    print(cost_matrix)
    all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
    print(f'Your code returned tours: {all_tours}')
    assert len(all_tours) == k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

    tour_cost = 0
    for tour in all_tours:
        assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
        i = 0
        for j in tour[1:]:
            tour_cost += cost_matrix[i][j]
            i = j
        tour_cost += cost_matrix[i][0]

    print(f'Tour cost obtained by your code: {tour_cost}')
    #assert abs(tour_cost - 6) <= 0.001, f'Expected tour cost is 6, your code returned {tour_cost}'
    for i in range(1, n):
        is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
        assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'
    print('------')
print('test passed: 15 points')

Trial # 0
 n= 7, k=2
cost_matrix = 
[[None, 2.0515406361316013, 3.62992685338202, 1.5940816148020338, 1.126428868004155, 3.9393551977148054, 4.731620349715716], [0.39808745284066194, None, 4.390183249015606, 2.184112967838536, 1.9515070022145036, 2.4677142806420442, 0.6977549529588534], [3.914548174086903, 2.582426276100081, None, 4.359077138230106, 1.3594252024650089, 0.03991432629875169, 2.7162791840387204], [4.102204619219774, 4.883541830272021, 4.5908141886560685, None, 3.506180966815455, 0.9754394292289725, 1.3313848964631152], [0.7521990642180543, 1.240184300746171, 2.8103301940369, 1.906005152984331, None, 4.877140111143436, 3.276210913144759], [2.7983788969734427, 2.8695228396416272, 1.6247863924373163, 1.30774369489707, 0.7723095812504382, None, 2.5062099179912307], [0.9933970420843358, 4.063550340092833, 3.345128388510288, 1.4363327341474403, 2.043659215626266, 3.747745249231787, None]]
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - 

## Problem 1 B

Notice that in previous part, it happens that with $k=4$ salespeople, we actually get a worse cost than using $k=3$ people. You can try out a few examples to convince yourself as to why this happens. 

We wish to modify the problem to allow salespeople to idle. In other words, although we input $k$ salespeople, the tour we construct may involve $1 \leq l \leq k$ salespeople. 

Modify the ILP formulation from the previous problem to solve the problem of up to $k$ people rather than exactly $k$ salespeople. Note that we still require that every vertex be visited exactly once by some salesperson. 

Complete the implementation of the function `upto_k_tsp_mtz_encoding(n, k, cost_matrix)` below. It follows the same input convention as previous problem but note that we are now computing a tour with at most $k$ salespeople. In other words, not all salespeople need be employed in the tour.

Your code must return a list `lst` that has less than or equal to  $k$ lists, wherein `lst[j]` represents the locations visited by the $j^{th}$ salesperson. 

For Example-1 from the previous part above, for $k=2$ or $k=3$, your code must return 
~~~
[ [0, 3, 1, 4, 2] ]
~~~
As it turns out, in this example a single salesperson suffices to yield optimal cost.

In [7]:
from pulp import *
def upto_k_tsp_mtz_encoding(n, k, cost_matrix):
    # check inputs are OK
    assert 1 <= k < n
    assert len(cost_matrix) == n, f'Cost matrix is not {n}x{n}'
    assert all(len(cj) == n for cj in cost_matrix), f'Cost matrix is not {n}x{n}'
    prob = LpProblem('kTSP', LpMinimize)
    # finish your implementation here
    # your code must return a list of k-lists [ [0, i1, ..., il], [0, j1,...,jl],...] wherein 
    # the ith entry in our list of lists represents the 
    # tour undertaken by the ith salesperson
    # your code here
    res = int(1e9), []
    for i in range(1,k+1):
        val = k_tsp_mtz_encoding(n, i, cost_matrix)
        value = 0
        for i in val:
            value += cost_matrix[i[-1]][0]
            value += cost_matrix[0][i[1]]
            for j in range(1, len(i)):
                value += cost_matrix[i[j-1]][i[j]]
        res = (value, val) if value < res[0] else res
    print(res)
    return res[1]
    raise NotImplementedError

In [8]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
k=3
all_tours = upto_k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) <= k, f'<= {k} tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

assert len(all_tours) == 1, f'In this example, just one salesperson is needed to optimally visit all vertices. Your code returns {len(all_tours)}'
print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 10) <= 0.001, f'Expected tour cost is 10, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 3 points')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/alimohammed1624/.virtualenvs/temp/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/nz/m7btrx796jl1bvgth7q4y5wm0000gn/T/fa2a1b7d12b14058a540f509b6e21536-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/nz/m7btrx796jl1bvgth7q4y5wm0000gn/T/fa2a1b7d12b14058a540f509b6e21536-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 32 COLUMNS
At line 200 RHS
At line 228 BOUNDS
At line 258 ENDATA
Problem MODEL has 27 rows, 29 columns and 89 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 9.4 - 0.00 seconds
Cgl0002I 5 variables fixed
Cgl0003I 0 fixed, 4 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 22 rows, 24 columns (24 integer (20 of which binary)) and 76 elements
Cutoff increment increased from 1e-05 to 0.9999
C

In [9]:
cost_matrix = [ 
 [None, 1, 1, 1, 1, 1, 1, 1],
    [0, None, 1, 2, 1, 1, 1, 1],
    [1, 0, None, 1, 2, 2, 2, 1],
    [1, 2, 2, None, 0, 1, 2, 1],
    [1, 1, 1, 1, None, 1, 1, 1],
    [0,  1, 2, 1, 1, None, 1, 1],
    [1, 0,  1, 2, 2, 2,None, 1],
    [1, 2, 2, 0, 1, 2, 1, None],
]
n = 8
k = 5

all_tours = upto_k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) <= k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 4) <= 0.001, f'Expected tour cost is 4, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 3 points')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/alimohammed1624/.virtualenvs/temp/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/nz/m7btrx796jl1bvgth7q4y5wm0000gn/T/fac2da2e1ef6470ea118e334c18e8010-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/nz/m7btrx796jl1bvgth7q4y5wm0000gn/T/fac2da2e1ef6470ea118e334c18e8010-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 71 COLUMNS
At line 524 RHS
At line 591 BOUNDS
At line 663 ENDATA
Problem MODEL has 66 rows, 71 columns and 260 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 4 - 0.00 seconds
Cgl0002I 8 variables fixed
Cgl0003I 0 fixed, 7 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 58 rows, 63 columns (63 integer (56 of which binary)) and 238 elements
Cutoff increment increased from 1e-05 to 0.9999
C

In [10]:
from random import uniform, randint

def create_cost(n):
    return [ [uniform(0, 5) if i != j else None for j in range(n)] for i in range(n)]

for trial in range(20):
    print(f'Trial # {trial}')
    n = randint(5, 11)
    k = randint(2, n//2)
    print(f' n= {n}, k={k}')
    cost_matrix = create_cost(n)
    print('cost_matrix = ')
    print(cost_matrix)
    all_tours = upto_k_tsp_mtz_encoding(n, k, cost_matrix)
    print(f'Your code returned tours: {all_tours}')
    assert len(all_tours) <= k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

    tour_cost = 0
    for tour in all_tours:
        assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
        i = 0
        for j in tour[1:]:
            tour_cost += cost_matrix[i][j]
            i = j
        tour_cost += cost_matrix[i][0]

    print(f'Tour cost obtained by your code: {tour_cost}')
    #assert abs(tour_cost - 6) <= 0.001, f'Expected tour cost is 6, your code returned {tour_cost}'
    for i in range(1, n):
        is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
        assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'
    print('------')
print('test passed: 4 points')

Trial # 0
 n= 7, k=3
cost_matrix = 
[[None, 3.6834164204008033, 3.272155991982506, 1.3354297711555403, 4.145752597555902, 0.22669276913058478, 4.180232420146122], [0.9460131817110107, None, 3.684038578499802, 4.093408504273987, 2.312195226502039, 4.931064028793817, 0.05208771623483821], [3.1557910369514586, 1.416444178685166, None, 2.158863140404404, 0.982627496159405, 2.09166989445245, 3.792975939025134], [1.8784992673257732, 1.8929298070524114, 1.708620595660214, None, 4.298122784129902, 3.330323726515335, 2.667904838236383], [4.7742912185631035, 1.686217368024926, 4.710558432856036, 1.8900462768273474, None, 2.101520074276777, 1.1356791262130046], [1.5529719716562744, 4.8695401919549415, 1.51257211736482, 4.855670164269723, 3.543640992933057, None, 0.5740731732053966], [2.4249672967042617, 4.9734551500417075, 4.676834253623373, 1.2565423997958987, 0.9299142651681525, 2.2451760887939063, None]]
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - 

## Problem 2 (10 points)

We noted the use of Christofides algorithm for metric TSP. We noted that for non-metric TSPs it does not work. 
In fact, the shortcutting used in Christofides algorithm can be _arbitrarily_ bad for a TSP that is symmetric but fails to be a metric TSP.

In this example, we would like you to frame a symmetric TSP instance ($C_{ij} = C_{ji}$) with $5$ vertices wherein the algorithm obtained by "shortcutting" the minimum spanning tree (MST), that would be a 2-factor approximation for metric TSP, yields an answer that can be quite "far off" from the optimal solution.

Enter a __symmetric__ cost-matrix for the TSP below as a 5x5 matrix as a list of lists following convention in our notes. such that the optimal answer is at least $10^6$ times smaller than that obtained by the TSP-based approximation. We will test your answer by running the TSP with shortcutting algorithm.

__Hint:__ Force the edges $(0,1), (1,2), (2,3)$ and $(3,4)$ to be the minimum spanning tree. But make the weight of the edge form $4$ back to $0$ very high.


__Note:__ this problem is tricky and requires you to be very familiar with how Christofides algorithm works. It may be wise to attempt the remaining problems first before this one. Do not worry about the diagonal entry of your matrices.


In [11]:
# Write down the cost matrix as a list of lists. Remember, we want an example with $5$ vertices
# cost_matrix = [[..], [...],[...]]
# The cost matrix should represent a TSP instance such that the optimal answer is 10^6 smaller than the 
# answer obtained by shortcutting a minimum spanning tree.
# your code here
cost_matrix = [[None, 1, 2, 2, int(1e9)],
               [1, None, 1, 2, 2],
               [2, 1, None, 1, 2],
               [2, 2, 1, None, 1],
               [int(1e9), 2, 2, 1, None]]

In [12]:
# check that the cost matrix is symmetric.
assert len(cost_matrix) == 5, f'Cost matrix must have 5 rows. Yours has {len(cost_matrix)} rows'
assert all(len(cj) == 5 for cj in cost_matrix), f'Each row of the cost matrix must have 5 entries.'
for i in range(5):
    for j in range(i):
        assert cost_matrix[i][j] == cost_matrix[j][i], f'Cost matrix fails to be symmetric at entries {(i,j)} and {(j,i)}'
print('Structure of your cost matrix looks OK (3 points).')

Structure of your cost matrix looks OK (3 points).


Please ensure that you run the two cells below or else, your tests will fail.

In [13]:
# MST based tsp approximation
import networkx as nx

# This code implements the simple MST based shortcutting approach that would yield factor of 2
# approximation for metric TSPs.
def minimum_spanning_tree_tsp(n, cost_matrix):
    G = nx.Graph()
    for i in range(n):
        for j in range(i):
            G.add_edge(i, j, weight=cost_matrix[i][j])
    T = nx.minimum_spanning_tree(G)
    print(f'MST for your graph has the edges {T.edges}')
    mst_cost = 0
    mst_dict = {} # store mst as a dictionary
    for (i,j) in T.edges:
        mst_cost += cost_matrix[i][j]
        if i in mst_dict:
            mst_dict[i].append(j)
        else:
            mst_dict[i] = [j]
        if j in mst_dict:
            mst_dict[j].append(i)
        else:
            mst_dict[j] = [i]
    print(f'MST cost: {mst_cost}')
    print(mst_dict)
    # Let's form a tour with short cutting
    def traverse_mst(tour_so_far, cur_node):
        assert cur_node in mst_dict
        next_nodes = mst_dict[cur_node]
        for j in next_nodes:
            if j in tour_so_far:
                continue
            tour_so_far.append(j)
            traverse_mst(tour_so_far, j)
        return
    tour = [0]
    traverse_mst(tour, 0)
    i = 0
    tour_cost = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]
    return tour, tour_cost

In [14]:
# optimal TSP tour taken from our notes using MTZ encoding
from pulp import *

def mtz_encoding_tsp(n, cost_matrix):
    assert len(cost_matrix) == n, f'Cost matrix is not {n}x{n}'
    assert all(len(cj) == n for cj in cost_matrix), f'Cost matrix is not {n}x{n}'
    # create our encoding variables
    binary_vars = [ # add a binary variable x_{ij} if i not = j else simply add None
        [ LpVariable(f'x_{i}_{j}', cat='Binary') if i != j else None for j in range(n)] 
        for i in range(n) ]
    # add time stamps for ranges 1 .. n (skip vertex 0 for timestamps)
    time_stamps = [LpVariable(f't_{j}', lowBound=0, upBound=n, cat='Continuous') for j in range(1, n)]
    # create the problem
    prob = LpProblem('TSP-MTZ', LpMinimize)
    # create add the objective function 
    objective_function = lpSum( [ lpSum([xij*cj if xij != None else 0 for (xij, cj) in zip(brow, crow) ])
                           for (brow, crow) in zip(binary_vars, cost_matrix)] )
    
    prob += objective_function 
    
    # add the degree constraints
    for i in range(n):
        # Exactly one leaving variable
        prob += lpSum([xj for xj in binary_vars[i] if xj != None]) == 1
        # Exactly one entering
        prob += lpSum([binary_vars[j][i] for j in range(n) if j != i]) == 1
    # add time stamp constraints
    for i in range(1,n):
        for j in range(1, n):
            if i == j: 
                continue
            xij = binary_vars[i][j]
            ti = time_stamps[i-1]
            tj = time_stamps[j -1]
            prob += tj >= ti + xij - (1-xij)*(n+1) # add the constraint
    # Done: solve the problem
    status = prob.solve(PULP_CBC_CMD(msg=False)) # turn off messages
    assert status == constants.LpStatusOptimal, f'Unexpected non-optimal status {status}'
    # Extract the tour
    tour = [0]
    tour_cost = 0
    while len(tour) < n:
        i = tour[-1]
        # find all indices j such that x_ij >= 0.999 
        sols = [j for (j, xij) in enumerate(binary_vars[i]) if xij != None and xij.varValue >= 0.999]
        assert len(sols) == 1, f'{sols}' # there better be just one such vertex or something has gone quite wrong
        j = sols[0] # extract the lone solutio 
        tour_cost = tour_cost + cost_matrix[i][j] # add to the tour cost
        tour.append(j) # append to the tour
        assert j != 0
    i = tour[-1]
    tour_cost = tour_cost + cost_matrix[i][0]
    return tour, tour_cost
        

In [15]:
#test that exact answer is 10^6 times smaller than approximate answer.
# compute MST based approximation
tour, tour_cost = minimum_spanning_tree_tsp(5, cost_matrix)
print(f'MST approximation yields tour is {tour} with cost {tour_cost}')
# compute exact answer
opt_tour, opt_tour_cost = mtz_encoding_tsp(5, cost_matrix)
print(f'Optimal tour is {opt_tour} with cost {opt_tour_cost}')
# check that the fraction is 1million times apart.
assert tour_cost/opt_tour_cost >= 1E+06, 'The TSP + shortcutting tour must be at least 10^6 times costlier than optimum. In your case, the ratio is {tour_cost/opt_tour_cost}'
print('Test passed: 7 points')

MST for your graph has the edges [(1, 0), (1, 2), (2, 3), (3, 4)]
MST cost: 4
{1: [0, 2], 0: [1], 2: [1, 3], 3: [2, 4], 4: [3]}
MST approximation yields tour is [0, 1, 2, 3, 4] with cost 1000000004


Optimal tour is [0, 2, 3, 4, 1] with cost 7
Test passed: 7 points


## Problem 3

In this problem, we wish to solve TSP with additional constraints. Suppose we are given a TSP instance in the form of a $n\times n$ matrix $C$ representing a complete graph. 

We wish to solve a TSP but with additional constraints specified as a list $[(i_0, j_0), \ldots, (i_k, j_k)]$ wherein each pair $(i_l, j_l)$ in the list specifies that vertex $i_l$ must be visited in the tour before vertex $j_l$. Assume that the tour starts/ends at vertex $0$ and none of the vertices in the constraint list is $0$. I.e, $i_l\not= 0, j_l \not= 0$ for all $0 \leq l \leq k$.

Modify one of the ILP encodings we have presented to solve TSP with extra constraints. Implement your solution in the function `tsp_with_extra_constraints(n, cost_matrix, constr_list)` where the extra argument `constr_list` is a list of pairs `[(i0,j0),...., (ik, jk)]` that specify for each pair `(il,jl)` that vertex `il` must be visited before `jl`. Assume that the problem is feasible (no need to handle infeasible instances). 
Your code should output the optimal tour as a list.

## Example

Consider again the graph with $5$ nodes and the following cost matrix from problem 1:

$$ \begin{array}{c|ccccc}
  \text{Vertices} & 0 & 1 & 2 & 3 & 4 \\ 
   \hline
 0 & - & 3 & 4 & 3 & 5 \\ 
 1 & 1 & - & 2 & 4 & 1 \\ 
 2 & 2 & 1 & - & 5 & 4 \\ 
 3 & 1 & 1 & 5 & - & 4 \\ 
 4 & 2 & 1 & 3 & 5 & - \\ 
 \end{array}$$
 
The optimal TSP tour will be $[0, 3, 1, 4, 2]$ with total cost $10$.

Suppose we added the constraints $[(4, 3), (1, 2)]$ we note that the tour satisfies the constraint $(1, 2)$ since it visits vertex $1$ before vertex $2$ but it unfortunately, $(4,3)$ is violated since vetex $3$ is visited before $4$ in the tour.


In [16]:
from pulp import *
def tsp_with_extra_constraints(n, cost_matrix, constraints):
    assert len(cost_matrix) == n, f'Cost matrix is not {n}x{n}'
    assert all(len(cj) == n for cj in cost_matrix), f'Cost matrix is not {n}x{n}'
    assert all( 1 <= i < n and 1 <= j < n and i != j for (i,j) in constraints)
    # TODO: encode the problem in pulp (a) decision variables; (b) constraints; (c) objective; (d) solve and extract
    #       solution. This is going to be very close to the MTZ encoding that we have presented in our notes. You can use
    #       our code as a starting point.
    # your code here
    
    prob = LpProblem('TSP', LpMinimize)

    dVars = [[[LpVariable(f'x{i}_{j}', lowBound=0, upBound=1, cat="Integer") for j in range(n)] for i in  range(n)],[LpVariable(f't{i}', lowBound=0, cat="Integer") for i in range(n)]]

    # Objective function
    prob += lpSum([cost_matrix[i][j] * dVars[0][i][j] if i!=j else None for i in range(n) for j in range(n)])

    # Constraints
    for i in range(n):
        prob += lpSum([dVars[0][i][j] for j in range(n)]) == 1
        prob += lpSum([dVars[0][j][i] for j in range(n)]) == 1

    for i in range(n):
        for j in range(n):
            if i!=j and i!=0 and j!=0:
                prob += dVars[1][i] - dVars[1][j] + dVars[0][i][j] - (n - 1) * (1 - dVars[0][i][j]) <= 0
            elif i == j:
                prob += dVars[0][i][j] == 0

    for i,j in constraints:
        prob += dVars[1][j] - dVars[1][i] >= 1

    prob.solve()

    vertices = [i for i in range(n)]
    j = 0
    tour = [0]
    while len(vertices) > 0:
        for l in vertices:
            if dVars[0][j][l].varValue == 1:
                if l!=0: tour.append(l)
                vertices.remove(l)
                j = l
                break
        if j == 0:
            break
    return tour
    raise NotImplementedError
       

In [17]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
constraints = [(3,4),(1,2)]
tour = tsp_with_extra_constraints(n, cost_matrix, constraints)
i = 0
tour_cost = 0
for j in tour[1:]:
    tour_cost += cost_matrix[i][j]
    i = j
tour_cost += cost_matrix[i][0]
print(f'Tour:{tour}')
print(f'Cost of your tour: {tour_cost}')
assert abs(tour_cost-10) <= 0.001, 'Expected cost was 10'
for i in range(n):
    num = sum([1 if j == i else 0 for j in tour])
    assert  num == 1, f'Vertex {i} repeats {num} times in tour'
for (i, j) in constraints:
    assert tour.index(i) < tour.index(j), f'Tour does not respect constraint {(i,j)}'
print('Test Passed (3 points)')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/alimohammed1624/.virtualenvs/temp/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/nz/m7btrx796jl1bvgth7q4y5wm0000gn/T/f719483ac1b44a6293fbc393f161440a-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/nz/m7btrx796jl1bvgth7q4y5wm0000gn/T/f719483ac1b44a6293fbc393f161440a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 34 COLUMNS
At line 208 RHS
At line 238 BOUNDS
At line 268 ENDATA
Problem MODEL has 29 rows, 29 columns and 95 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 9.5 - 0.00 seconds
Cgl0002I 5 variables fixed
Cgl0003I 0 fixed, 4 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 24 rows, 24 columns (24 integer (20 of which binary)) and 80 elements
Cutoff increment increased from 1e-05 to 0.9999
C

In [18]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
constraints = [(4,3),(1,2)]
tour = tsp_with_extra_constraints(n, cost_matrix, constraints)
i = 0
tour_cost = 0
for j in tour[1:]:
    tour_cost += cost_matrix[i][j]
    i = j
tour_cost += cost_matrix[i][0]
print(f'Tour:{tour}')
print(f'Cost of your tour: {tour_cost}')
assert abs(tour_cost-13) <= 0.001, 'Expected cost was 13'
for i in range(n):
    num = sum([1 if j == i else 0 for j in tour])
    assert  num == 1, f'Vertex {i} repeats {num} times in tour'
for (i, j) in constraints:
    assert tour.index(i) < tour.index(j), f'Tour does not respect constraint {(i,j)}'
print('Test Passed (3 points)')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/alimohammed1624/.virtualenvs/temp/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/nz/m7btrx796jl1bvgth7q4y5wm0000gn/T/54e73994a7f3457caffbe88179a7bf74-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/nz/m7btrx796jl1bvgth7q4y5wm0000gn/T/54e73994a7f3457caffbe88179a7bf74-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 34 COLUMNS
At line 208 RHS
At line 238 BOUNDS
At line 268 ENDATA
Problem MODEL has 29 rows, 29 columns and 95 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 9.5 - 0.00 seconds
Cgl0002I 5 variables fixed
Cgl0003I 0 fixed, 4 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 24 rows, 24 columns (24 integer (20 of which binary)) and 80 elements
Cutoff increment increased from 1e-05 to 0.9999
C

In [19]:
from random import uniform, randint

def create_cost(n):
    return [ [uniform(0, 5) if i != j else None for j in range(n)] for i in range(n)]

for trial in range(20):
    print(f'Trial # {trial}')
    n = randint(6, 11)
    cost_matrix = create_cost(n)
    constraints = [(1, 3), (4, 2), (n-1, 1), (n-2, 2)]
    tour = tsp_with_extra_constraints(n, cost_matrix, constraints)
    i = 0
    tour_cost = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]
    print(f'Tour:{tour}')
    print(f'Cost of your tour: {tour_cost}')
    for i in range(n):
        num = sum([1 if j == i else 0 for j in tour])
        assert  num == 1, f'Vertex {i} repeats {num} times in tour'
    for (i, j) in constraints:
        assert tour.index(i) < tour.index(j), f'Tour does not respect constraint {(i,j)}'
print('Test Passed (10 points)')

Trial # 0
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/alimohammed1624/.virtualenvs/temp/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/nz/m7btrx796jl1bvgth7q4y5wm0000gn/T/10488fd54f0447d99fd1b4ca77d91745-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/nz/m7btrx796jl1bvgth7q4y5wm0000gn/T/10488fd54f0447d99fd1b4ca77d91745-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 47 COLUMNS
At line 306 RHS
At line 349 BOUNDS
At line 391 ENDATA
Problem MODEL has 42 rows, 41 columns and 146 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 9.35434 - 0.00 seconds
Cgl0002I 6 variables fixed
Cgl0003I 0 fixed, 5 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 35 rows, 35 columns (35 integer (30 of which binary)) and 126 elements
Cbc0038I Initial state - 4 intege

## Answers to Select Problems

### 1A part A

- Vertex 0: $k$ edges leave and $k$ edges enter.
- Vertex 1, ..., n-1: 1 edge leaves and 1 edge enters (same as TSP).

### 1A part B

This is a trick question. There is no need to change any of the time stamp related constraints.


## That's All Folks!